# Import

In [2]:
import pandas as pd
import numpy as np
from importlib import reload
from astropy.io import fits
from forecast_SN_GW import get_hubblefit

# Load data

In [3]:
data_path = '../'

In [4]:
d_ps1 = pd.read_csv(data_path+'Data/SN_ps_snls_sdss_hst.csv', sep=' ', index_col='CID')
ps1_pd = d_ps1[d_ps1['IDSURVEY'] == 15]

d_X = pd.read_csv(data_path+'Data/jla_lcparams.txt', sep=' ', index_col='#name')

bias_data = fits.getdata(data_path+'Data/covmat/C_bias.fits')
cal_data = fits.getdata(data_path+'Data/covmat/C_cal.fits')
dust_data = fits.getdata(data_path+'Data/covmat/C_dust.fits')
host_data = fits.getdata(data_path+'Data/covmat/C_host.fits')
model_data = fits.getdata(data_path+'Data/covmat/C_model.fits')
nonia_data = fits.getdata(data_path+'Data/covmat/C_nonia.fits')
pecvel_data = fits.getdata(data_path+'Data/covmat/C_pecvel.fits')
stat_data = fits.getdata(data_path+'Data/covmat/C_stat.fits')
covX = bias_data + cal_data + dust_data + host_data + model_data + nonia_data + pecvel_data + stat_data

# Reproduce JLA results

## Build data array for JLA fit

In [5]:
mass = np.zeros_like(d_X['3rdvar'].values)
for i, m in enumerate(d_X['3rdvar'].values):
    if m > 10:
        mass[i] = -1.
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, mass])

d_sigma = pd.read_csv(data_path+'Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## JLA fit

In [6]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])

In [8]:
g.fit()
g.minuit.values

------------------------------------------------------------------
| FCN = 682.9                   |     Ncalls=159 (159 total)     |
| EDM = 1.83E-05 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------


<ValueView of Minuit at 5624063c2948>
  Mb: -19.048500801549057
  alpha1: 0.1412273345463002
  alpha2: -3.101541148875581
  alpha3: -0.06995742381361028
  omgM: 0.295371105743009

In [20]:
g.minuit.errors

<ErrorView of Minuit at 558dbd2ab328>
  Mb: 0.023170305107844093
  alpha1: 0.006590085152848236
  alpha2: 0.08072799314194437
  alpha3: 0.023110783961262468
  omgM: 0.03393357277465823

# Fit with linear correction in redshift 

## Build data array 

In [9]:
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, d_X['zcmb'].values])

d_sigma = pd.read_csv(data_path+'Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## Fit 

In [10]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])
g.fit()

------------------------------------------------------------------
| FCN = 691.2                   |     Ncalls=204 (204 total)     |
| EDM = 5.18E-05 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------


(0.15357536939936012, 0.0050001990226806676)

In [11]:
g.minuit.values

<ValueView of Minuit at 5624062bc448>
  Mb: -19.06817843629274
  alpha1: 0.1362530145217983
  alpha2: -3.1018762981565584
  alpha3: -0.22255112915467262
  omgM: 0.4697946442653904

In [12]:
g.minuit.errors

<ErrorView of Minuit at 5624062bc448>
  Mb: 0.023213045494070748
  alpha1: 0.006379329236516696
  alpha2: 0.08060274502331118
  alpha3: 0.19386062016323577
  omgM: 0.17517034845614787